In [ ]:
import os
import pandas as pd
import string
from collections import Counter
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
def concatenate_files():
    """
    Concatenates files of data of 100 instances together into 1 dataframe and saves it to a tsv
    """
    cwd = os.path.abspath('') 
    folders = os.listdir(path+'bijlagen') 

    dfs = []
    for folder in folders:
        file = os.listdir(path+'/bijlagen/'+folder) 
        df = pd.read_excel(path+'/bijlagen/'+folder+'/'+file[0], engine='openpyxl') 
        num += len(df)
        dfs.append(df)
    df = pd.concat(dfs)
    df.to_csv('data/data-2010-2020.tsv', sep='\t', index=False)

In [ ]:
def base_clean_data(df):
    
    # Remove irrelevant and duplicate columns
    df_no_irr_cols = df.drop(['Publicatie.1', 'Publicatietype.1', 'Sectie', 'Lengte.1', 'Cite', 'Bedrijf', 'Agg-copyright', 'Pub-copyright', 'Titel.1', 'Weergeven', 'Ticker'], axis=1)

    # Remove rows without text 
    df_not_na = df_no_irr_cols[~df_no_irr_cols['Hlead'].isna()]
    print('# Removed empty: ', len(df)-len(df_not_na))
    
    # Remove duplicates
    df_no_dups = df_not_na.drop_duplicates()
    
    print('# Removed duplicates after removal empty: ', len(df_not_na)-len(df_no_dups))
    return df_no_dups

In [ ]:
def extract_year(df):
    # Remove punctuation
    df['Jaar'] = df['Datum'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

    # Retrieve years from various date formats
    df['Jaar'] = df['Jaar'].apply(lambda x: x.rsplit(' ', 1)[0] if len(x.split()) >= 4 else x)
    df['Jaar'] = df['Jaar'].apply(lambda x: x.split()[2] if len(x.split()) > 2 else x.split()[1])
    
    return df

In [ ]:
def general_stats(df, path):
    print(df.isna().mean().round(4) * 100)
    
    top_auteurs = df['Auteur'].value_counts().head(10)
    top_publicaties = df['Publicatie'].value_counts().head(10)

    jaar = Counter(df['Jaar'])
    top_jaar = pd.Series(dict(sorted(jaar.items())), name='Jaar')

    print(top_auteurs)     
    print(top_publicaties)  
    print(top_jaar)
    
    plt.figure()
    ta = top_auteurs.plot(kind='bar')
    ta.axhline(np.array(top_auteurs.values).mean(), color='r', linestyle='--', lw=1, label='mean count')
    plt.xlabel('authors')
    plt.ylabel('number of articles')
    plt.legend(loc='upper right')
    plt.savefig(path + 'top-auteurs', bbox_inches = 'tight')
    
    plt.figure()
    tp = top_publicaties.plot(kind='bar')
    tp.axhline(np.array(top_publicaties.values).mean(), color='r', linestyle='--', lw=1, label='mean count')
    plt.xlabel('publications')
    plt.ylabel('number of articles')
    plt.legend(loc='upper right')
    plt.savefig(path + 'top-publicaties', bbox_inches = 'tight')
    
    plt.figure()
    tj = top_jaar.plot(kind='bar')
    tj.axhline(np.array(top_jaar.values).mean(), color='r', linestyle='--', lw=1, label='mean count')
    plt.xlabel('years')
    plt.ylabel('number of articles')
    plt.legend(loc='upper right')
    plt.savefig(path + 'top-jaar', bbox_inches = 'tight')

In [ ]:
def text_length_stats(df_col, path):    
    # Get list of word lengths of data
    sv = []
    reg = []
    for i in df_col:
        if 'SAMENVATTING' in ' '.join(i.split()[:3]):
            sv.append(len(i.split()))
        else:
            reg.append(len(i.split()))
            
    sv_avg = sum(sv)/len(sv)
    reg_avg = sum(reg)/len(reg)
    
    print("Number of samenvattingen: ", len(sv))
    print("Number of regular texts: ", len(reg))
    print("Avg word count samenvattingen: ", sv_avg)
    print("Avg word count regular texts: ",reg_avg)

    plt.figure()
    plt.hist(reg, bins=50, alpha=0.3, label='regular texts')
    plt.hist(sv, bins=50, alpha=1, label='samenvatting')
    plt.axvline(np.array(reg).mean(), color='r', linestyle='dashed', linewidth=1, label='mean regular texts')
    plt.axvline(np.array(sv).mean(), color='k', linestyle='dashed', linewidth=1, label='mean samenvatting')
    
    plt.xlabel("word count")
    plt.ylabel("number of articles")
    plt.legend(loc='upper right')

    plt.savefig(path + 'wc-sv-rt-ratio.png', bbox_inches = 'tight')
    print('Figure saved in ' + path + 'wc-sv-rt-ratio.png')

In [ ]:
def remove_samenvattingen(base_df):
    base_df_regular = base_df[base_df['Tekstsoort'] == 'regular']
    return base_df_regular.drop(['Tekstsoort'], axis=1)

In [ ]:
# concatenate_files()
df = pd.read_csv('data/data-2010-2020.tsv', sep='\t')

print('Length original data: ', len(df))
print('')

os.makedirs('plots', exist_ok=True)

# Data cleaning
print('Data cleaning:')
os.makedirs('plots/base_clean', exist_ok=True)

bc_path = 'plots/base_clean/'
base_df = base_clean_data(df)
base_df = extract_year(base_df)    

# Statistics base clean
general_stats(base_df, bc_path)
text_length_stats(base_df['Hlead'], bc_path)

In [ ]:
# Remove summaries
base_df['Tekstsoort'] = base_df.Hlead.apply(lambda x: 'samenvatting' if 'SAMENVATTING' in ' '.join(x.split()[:3]) else 'regular')
regular_base_df = remove_samenvattingen(base_df)
print("Stats after removing samenvattingen:")
print(regular_base_df.isna().mean().round(4) * 100)

In [ ]:
def clean_author(text):
    text = text.lower()
    if text[0] != ';':
        text = text.split(";", 1)[0]
    else:
        text = ' '.join(text.split(";", 1)[1:])
    if text == 'een':
        text = 'door een verslaggever'
    text = text.split("/", 1)[0]
    text = text.split("e-mail", 1)[0]
    text = re.sub('door', '', text, flags=re.IGNORECASE)
    text = re.sub("[()]", '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub('|', '', text)
    text = ' '.join([x for x in text.split() if "@" not in x])
    
    return text

def get_unique_authors(df):
    authors = []
    for i in set(df['Auteur'].values):
        if isinstance(i, str): 
            i = clean_author(i)
            if len(i) > 0:
                authors.append(i)
    return list(set(authors))

def find_author(text, author, regex):
    # if author col is empty
    if pd.isnull(author):
        if re.match(regex, text, flags=re.IGNORECASE):
            new_author = re.search(regex, text, flags=re.IGNORECASE).group(0)
            author = clean_author(new_author)
    else:
        author = author.capitalize()
    return author

In [ ]:
def remove_word_from_text(text, regex, flags):
    if flags == True:
        text = re.sub(regex, '', text, flags=re.IGNORECASE)
    else:
        text = re.sub(regex, '', text)        
    return text

In [ ]:
def remove_place(text, cities_reg):
    if len(text.split()) > 0:
        removed_part = remove_word_from_text(' '.join(text.split()[:2]), cities_reg, True)
        text = removed_part + ' ' + ' '.join(text.split()[2:])
    if '-' in "".join(text.split()[:5]):
        text = text.split('-', 1)[1]
    return text

In [ ]:
def remove_extra_words(regular_df, authors):
    vervolg = "(vervolg van vervolg van pagina\s\d{1,2})|(vervolg van pagina\s\d{1,2})|(vervolg vanpagina\s\d{1,2})|(vervolg van pagina\s\d{1,2}-)|(vervolg van voorpagina)|(vervolg van pagina)"
    city_list = pd.read_csv('data/Woonplaatsen_in_Nederland_2020_20122021_042012.csv', sep=';')['Woonplaatsen'].tolist()
    # Regex for authors
    authors_reg = '|'.join(authors)
    door_authors_reg = "(door) " + "(" + authors_reg + ")"
    door_authors_extra_reg =  "(" + authors_reg + ")" + '|' + "(" + door_authors_reg + ")" 
    cities_reg = '|'.join(city_list) # removed Ee and EEN which are places

    # Remove leading and trailing whitespaces
    regular_df['preprocessed_hlead'] = regular_df.Hlead.apply(lambda x: " ".join(x.split()))
    
     # Remove 'vervolg van pagina x'
    regular_df['preprocessed_hlead'] = regular_df.preprocessed_hlead.apply(lambda x: remove_word_from_text(x, vervolg, True)) 
    
    print("Vervolgpagina's removed")
    
    # Find new author
    regular_df['Nieuwe auteur'] = regular_df.apply(lambda x: find_author(x.preprocessed_hlead, x.Auteur, door_authors_extra_reg), axis=1)   
    
    print("New authors found")
    
    # Remove 'Authors/door Author/door Author-'
    regular_df['preprocessed_hlead'] = regular_df.preprocessed_hlead.apply(lambda x: remove_word_from_text(x, door_authors_extra_reg, True))   

    # Remove leading and trailing whitespaces
    regular_df['preprocessed_hlead'] = regular_df.preprocessed_hlead.apply(lambda x: " ".join(x.split()))   
    
    print("Authors removed from text")
    
    # Remove first -
    regular_df['preprocessed_hlead'] = regular_df.preprocessed_hlead.apply(lambda x: x[1:] if x[0]=='-' else x)   
    
    # Remove leading and trailing whitespaces
    regular_df['preprocessed_hlead'] = regular_df.preprocessed_hlead.apply(lambda x: " ".join(x.split()))  
    
    # Remove place name 
    regular_df['preprocessed_hlead'] = regular_df.preprocessed_hlead.apply(lambda x: remove_place(x, cities_reg))  
    
    # Remove leading and trailing whitespaces
    regular_df['preprocessed_hlead'] = regular_df.preprocessed_hlead.apply(lambda x: " ".join(x.split()))  
    print("Placenames removed")

    return regular_df

In [ ]:
# authors = get_unique_authors(regular_base_df)
# regular_df_cleaned = remove_extra_words(regular_base_df, authors)
# regular_df_cleaned.to_csv('complete-clean-preprocessed-data-2010-2020.tsv', sep='\t', index=False)
regular_df_cleaned = pd.read_csv('data/complete-clean-preprocessed-data-2010-2020.tsv', sep='\t')
print(regular_df_cleaned.isna().mean().round(4) * 100)

print(regular_df_cleaned['Auteur'].value_counts())
print(regular_df_cleaned['Nieuwe auteur'].value_counts())

In [ ]:
plt.figure()
top_new_auteurs = regular_df_cleaned['Nieuwe auteur'].value_counts().head(10)
ta = top_new_auteurs.plot(kind='bar')
ta.axhline(np.array(top_new_auteurs.values).mean(), color='r', linestyle='--', lw=1, label='mean count')
plt.xlabel('authors')
plt.ylabel('number of articles')
plt.legend(loc='upper right')
plt.savefig(bc_path + 'top-new-auteurs', bbox_inches = 'tight')
    